# Cadenas de Markov 

Un caminante aleatorio es un tipo particular de una *cadena de Markov*.

Una cadena de Markov es, a su vez, un tipo de 
*proceso estocástico*, es decir una evolución en 
el tiempo que no es determinista, sino donde hay 
una probabilidad dada, el sistema lleva a cabo 
una *transición* hacia un estado nuevo. Las cadenas de Markov son de gran importancia hoy día en las ciencias.

La particularidad de las cadenas de Markov es que satisfacen la *propiedad de Markov*: el estado nuevo depende *sólo de un estado anterior*. Se suele decir que el proceso *no tiene memoria*.

Por ejemplo, una caminata aleatoria satisface esto, ya que la posición al tiempo $t+1$ depende sólo de la posición al tiempo $t$ (no de posiciones a tiempos anteriores).  

Sea $X_n$ el estado del proceso al tiempo $n$. Nos interesaremos por el momento en procesos de Markov tal que
$P(X_{n+1} = j \, | \, X_n = i) = p_{ij}$, independiente del tiempo. La notación aquí es de una *probabilidad condicional*, es decir, la probabilidad de que el sistema está en el estado $j$ al tiempo $n+1$, *dado que* estaba en el estado $i$ al tiempo $n$.

[1] Las $p_{ij}$ forman una matriz. 

(i) Escribe la matriz para un caminante aleatorio con fronteras reflejantes en 0 y $L=5$, si tiene probabilidades $p$, $q$, y $r$ de brincar a la derecha o izquierda, o de permanecer en el mismo lugar, respectivamente.

(ii) ¿Qué propiedad debe satisfacer cualquier matriz de transición?

In [41]:
function generar_caminante_aleatorio(L::Int64,p::Float64=1/3,q::Float64=1/3,r::Float64=1/3)
    m=zeros(L,L)
    #Asingamos los valores sin las fronteras primero
    for i in 1:L
        m[i,i]=r
        if i<L
            m[i,i+1]=p
        end
        if i>1
            m[i,i-1]=q
        end
    end
    #Y ahora las fronteras
    m[1,1]+=q
    m[L,L]+=p
    return m
end
#Visualizamos
m=generar_caminante_aleatorio(6)

6x6 Array{Float64,2}:
 0.666667  0.333333  0.0       0.0       0.0       0.0     
 0.333333  0.333333  0.333333  0.0       0.0       0.0     
 0.0       0.333333  0.333333  0.333333  0.0       0.0     
 0.0       0.0       0.333333  0.333333  0.333333  0.0     
 0.0       0.0       0.0       0.333333  0.333333  0.333333
 0.0       0.0       0.0       0.0       0.333333  0.666667

La suma de los elementos de cualquiera de sus renglones o columnas debe de ser uno e.d.
$$ \sum_j^L p_{ij}= \sum_i^L p_{ij}=1$$

Tambien se cumple q la determinante de esta matriz es 0

[2] Considera una cadena que tiene 4 estados. Desde cada estado puede brincar a cada otro estado con igual probabilidad. 

**(i) Escribe la matriz de transición $\mathsf{P} = (p_{ij})$. ¿Qué propiedades tiene que satisfacer?**

In [2]:
P=ones(4,4)
P=P*1/4

4x4 Array{Float64,2}:
 0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25

Esta matriz satisface la propiedad añadida de que cada entrada debe valer "1 / (su numero de estados)".

Además, tambien su traza es 1.

(ii) ¿Qué tipo de propiedades nos podrían interesar?, suponiendo que esto modela un estado físico, químico o biológico que puede estar en uno de estos estados.

Es interesante q su determinante sea 0, pues no la podemos diagonalizar mas de lo que esta, lo que nos habla de que esto quiza no tiene modos normales.
Tambien es importante que, tomando en cuenta que la composición de esta función es una multiplicación de matrices, las potencias de estas matrices sigan manteniendo la propiedad de que la suma de todas las entradas es 1.
Esto quiere decir que el sistema se mantiene normalizado.

(iii) Simula la dinámica del sistema: escribe una función que genera una realización de cierta longitud.

In [25]:
function realizacion_Markov(inicio::Array{Float64,1}, matriz::Array{Float64,2}, longitud::Int64)
    if size(inicio)[1] != size(matriz)[1]
        return "Las dimensiones no coinciden"
    end
    return (matriz^longitud)*inicio
end

realizacion_Markov (generic function with 1 method)

In [37]:
dim = 11
inicio = zeros(dim)
inicio[(dim-1)/2]=1
matriz=generar_caminante_aleatorio(dim)
realizacion_Markov(inicio,matriz,2)

11-element Array{Float64,1}:
 0.0     
 0.0     
 0.111111
 0.222222
 0.333333
 0.222222
 0.111111
 0.0     
 0.0     
 0.0     
 0.0     

(iv) Escribe funciones que calculen las cantidades de interes numéricamente.

[3] Escribe una función para simular una realización de una trayectoria correspondiendo a una matriz de transición *arbitraria* $\mathsf{P}$ que satisfaga las propiedades de 2(i).

## Método estilo enumeración exacta

Tal como hicimos para un caminante aleatorio, para una cadena de Markov también podemos cambiar de punto de vista y estudiar la evolución de la *distribución de probabilidad*, la cual nos dice la probabilidad de que la cadena se encuentra en cada estado al tiempo $t$.

Primero veamos esto analíticamente.

[4] Supongamos que la cadena empieza en alguna distribución de probabilidad inicial $\mathbf{P}_0 = (P_{0,i}: i=1, \ldots, n)$, donde $n$ es el número de estados.

(i) Escribe la ecuación maestra que describe cómo evoluciona la probabilidad en 1 paso.  

(ii) Escríbela de nuevo, ahora usando notación matricial.

(ii) ¿Qué pasa en 2 pasos? ¿En $N$ pasos?

(i)La ecuacion es como la que vimos en la nota 4.
$P_{t+1}(i)=P_{t}(i-1)*p + P_{t}(i+1)*q + P_{t}(i)*r$
(ii) Asi que en notacion matricial sería $\vec{P_{t+1}} =
 \begin{pmatrix}
  r & q & 0 & 0 & \cdots & 0 \\
  p & r & q & 0 & \cdots & 0 \\
  0 & p & r & q & \cdots & 0 \\
  \vdots  & \vdots  & \vdots  & \ddots & \vdots  \\
  0 & 0 & 0 & 0 & \cdots & r 
 \end{pmatrix}\vec{P_t}$

Como vi antes (quiza debería de haberlo visto de otra forma pero no se me ocurre), debido a que la composicion de la aplicación de una transf lineal que es una matriz es multiplicacion de estas, en el tiempo n tendriamos: $$ \vec{P_{t+n}}= \vec{P_{t}}^n$$

[5] Simula la dinámica de la cadena de Markov de la pregunta 2 desde una condición inicial que es una delta en algún estado por un tiempo $t$. ¿Qué observas cuando $t \to \infty$?

In [43]:
matriz^1000

11x11 Array{Float64,2}:
 0.0909091  0.0909091  0.0909091  …  0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091  …  0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091     0.0909091  0.0909091  0.0909091
 0.0909091  0.0909091  0.0909091  …  0.0909091  0.0909091  0.0909091

Se puede ver que se tiende a la matriz que vimos en la pregunta 2(i), una matriz q representa la misma probabilidad de pasar de cualquier lugar a cualquier otro.

[6] Este mismo fenómeno (lo que pasa cuando $t \to \infty$) ocurre para "casi todas" las cadenas de Markov (cuando se satisfacen ciertas condiciones técnicas). Explícalo usando propiedades de la matriz que vieron en álgebra lineal. ¿A qué corresponde lo que pasa cuando $t \to \infty$?

In [4]:
function mover(x)
    null
end

mover (generic function with 2 methods)

In [3]:
function mover(x,y)
    x+y
end

mover (generic function with 2 methods)